# Open / Closed Principle (OCP)

> “Software entities (classes, modules, functions, etc.) should be **open for extension** but **closed for modification**.” – Bertrand Meyer

## 1. Concept
* **Open for extension** – we can add new behaviour without touching existing source code.
* **Closed for modification** – once a class is tested & released, its source stays intact.
* This applies to monoliths, **services**, and **microservices** alike; evolve through extension.

### Why it matters
* Minimises *ripple effects* when new requirements arrive.
* Preserves backward compatibility for callers.
* Encourages using **interfaces**, **abstract classes**, and GoF patterns (Strategy, Decorator, Factory, …).

## 2. Problematic design without OCP

In [1]:
// BAD: PaymentProcessor must change for every new payment method
enum PaymentType {
  CreditCard = "credit-card",
  PayPal = "paypal"
  // Next month: ApplePay, Crypto, etc.
}

class PaymentProcessor {
  process(amount: number, type: PaymentType) {
    switch (type) {
      case PaymentType.CreditCard:
        console.log(`Processing $${amount} through credit card API`);
        break;
      case PaymentType.PayPal:
        console.log(`Redirecting to PayPal for $${amount}`);
        break;
      default:
        throw new Error(`Unsupported payment type: ${type}`);
    }
  }
}

const legacy = new PaymentProcessor();
legacy.process(42, PaymentType.CreditCard);

Processing $42 through credit card API


### Issues
1. **Modification required** in `PaymentProcessor` (and the enum) for every new gateway.
2. **Switch explosion** – poor separation of concerns.
3. **High risk** of breaking existing payments when editing branching logic.

## 3. Refactor applying OCP (Strategy pattern)

In [2]:
// GOOD: add payment behaviours via new classes—no core modification
interface PaymentMethod {
  pay(amount: number): void;
}

class CreditCardPayment implements PaymentMethod {
  pay(amount: number) {
    console.log(`Charged $${amount} to credit‑card gateway`);
  }
}

class PayPalPayment implements PaymentMethod {
  pay(amount: number) {
    console.log(`Charged $${amount} via PayPal REST`);
  }
}

// Tomorrow's feature:
class CryptoPayment implements PaymentMethod {
  pay(amount: number) {
    console.log(`Paid $${amount} in BTC`);
  }
}

class PaymentProcessor {
  constructor(private method: PaymentMethod) {}

  checkout(amount: number) {
    this.method.pay(amount);
  }
}

// Client composes behaviour:
const cryptoOrder = new PaymentProcessor(new CryptoPayment());
cryptoOrder.checkout(99);

Paid $99 in BTC


### Benefits achieved
* `PaymentProcessor` **remains unchanged** while capabilities grow.
* New payment methods become **plug‑ins**, testable in isolation.
* Consumers choose behaviour via dependency injection or factories.

## 4. Trade‑offs
* **More classes / wiring** – increased complexity in tiny projects.
* Designing stable interfaces upfront is harder.
* Over‑abstracting early may lead to unnecessary indirection.

## 5. Spotting OCP violations – quick checklist
1. Do you see `switch`/`if‑else` keyed by a type flag?
2. Must you open old files whenever a new variation appears?
3. Do stable unit‑tests break when unrelated functionality is added?

## 6. Related GoF patterns
* **Strategy** – swap algorithms at runtime (shown above).
* **Decorator** – dynamically add responsibilities.
* **Template Method** – keep skeleton, vary steps.
* **Factory Method / Abstract Factory** – create objects without binding to concretes.

## 7. References
* Bertrand Meyer, *Object‑Oriented Software Construction* (1988)
* Robert C. Martin, *Agile Software Development: Principles, Patterns, and Practices* (2002)
* https://principles.dev/#ocp